In [1]:
import os
import numpy as np
import PIL
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import keras
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop
from keras import activations
from keras.preprocessing import image

from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


In [2]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

In [3]:
# Data augmentation and pre-processing

datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=[0.9, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

datagen_test = ImageDataGenerator(rescale=1./255)

In [4]:
# Directory setup

train_dir = '../all_train_data/'

input_shape = (224, 224)
batch_size = 32

In [5]:
# Making data generator
generator_train = datagen_train.flow_from_directory(
    directory=train_dir,
    target_size=input_shape,
    batch_size=batch_size,
    shuffle=True
)

Found 6252 images belonging to 5 classes.


In [6]:
# Setting epoch size for batch step calculation
epochs = 50

# Steps per epoch setup
steps_per_epoch = generator_train.n / batch_size

In [7]:
# joing directory and filenames
image_paths_train = path_join(train_dir, generator_train.filenames)

# classes available
cls_train = generator_train.classes

class_names = list(generator_train.class_indices.keys())
num_classes = generator_train.num_classes

In [8]:
# In case of unequal number of dataset, use class weight balancing
class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(cls_train),
    y=cls_train
)

### Create model

In [9]:
resnet = keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all the layers
# for layer in resnet.layers:
#     layer.trainable = False

model = Sequential()
 
# Add the resnet convolutional base model
model.add(resnet)
 
# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

/home/shashankwadhwa/env/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 5125      
Total para

In [10]:
# setting learning rate
optimizer = Adam(lr=1e-5)

# Loss definition
loss = 'categorical_crossentropy'

# Metrics for measurement
metrics = ['categorical_accuracy']

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8704049594368523298
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11273397863
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1304133640883138549
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [12]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [13]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Begin training for transfer learning
history = model.fit_generator(
    generator=generator_train,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    class_weight=class_weight
)

Epoch 1/50
196/195 [==============================] - 228s 1s/step - loss: 1.2264 - categorical_accuracy: 0.5462
Epoch 2/50
196/195 [==============================] - 204s 1s/step - loss: 0.7645 - categorical_accuracy: 0.7133
Epoch 3/50
196/195 [==============================] - 204s 1s/step - loss: 0.6096 - categorical_accuracy: 0.7723
Epoch 4/50
196/195 [==============================] - 203s 1s/step - loss: 0.5283 - categorical_accuracy: 0.7993
Epoch 5/50
196/195 [==============================] - 204s 1s/step - loss: 0.4873 - categorical_accuracy: 0.8174
Epoch 6/50
196/195 [==============================] - 204s 1s/step - loss: 0.4284 - categorical_accuracy: 0.8364
Epoch 7/50
196/195 [==============================] - 205s 1s/step - loss: 0.4181 - categorical_accuracy: 0.8421
Epoch 8/50
196/195 [==============================] - 205s 1s/step - loss: 0.3783 - categorical_accuracy: 0.8575
Epoch 9/50
196/195 [==============================] - 204s 1s/step - loss: 0.3457 - categorical_

KeyboardInterrupt: 

In [14]:
model.save('k4_sub.h5')

### Get predictions

In [1]:
from keras.models import load_model

Using TensorFlow backend.


In [2]:
model = load_model('k4_sub.h5')

In [17]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import os

datagen_test = ImageDataGenerator(rescale=1./255)
test_dir = '../test_data_2/'
input_shape = (224, 224)
batch_size = 32

test_generator = datagen_test.flow_from_directory(
    directory=test_dir,
    target_size=input_shape,
    batch_size=batch_size,
    shuffle=False
)

predictions = model.predict_generator(
    test_generator,
    steps=test_generator.samples/test_generator.batch_size,
    verbose=1
)
predicted_classes = pd.Series(np.argmax(predictions, axis=1))
predicted_labels = pd.Series([x+1 for x in predicted_classes])

Found 2680 images belonging to 1 classes.
84/83 [==============================] - 23s 278ms/step


In [20]:
sub_df = pd.DataFrame()
filenames = [x.split('/')[1] for x in test_generator.filenames]
sub_df['image'] = filenames
sub_df['category'] = predicted_labels
sub_df.to_csv('submission_k4.csv', index=False)